In [ ]:
import sys
import pandas as pd
import math
import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import allel
from sklearn import metrics

In [ ]:
#####COMPARE POWER OF OUTLIER AND NULL THRESHOLD APPROACHES USING INFERENCE RESULTS (OUTPUTS FROM get_H12_snp.py AND SF2)
#####AND FIXED FILE FROM SLIM OUTPUT.#####
winSize=10000
regionLen=997204
#Set col names for categorising FPs, TPs etc (adjust for SweepFinder2 analysis)
col1  = 'H12'
col2 = 'sweep'
#Create empty df for results
rdf = pd.DataFrame()
        
####GET HIGHEST H12 VALUE FROM SIMULATIONS WITH NO SWEEPS####
lst = []
for rep in range(1,201):
    try:
        df = pd.read_csv(inferenceFile, sep='\t', header=0)
        #Get max value of current replicaate
        lst.append(df['H12'].max())
    except Exception:
        pass
#Get max value of all resplicates
null_threshold = np.max(lst)

#Create empty list to store results for each 2Nes value
results = []
#Loop through 2Nes values
for N in ['noSweeps',100, 1000, 10000]:
    #Empty lists to store TP, FP etc for each replicate
    null = []
    tail = []
    #Loop through replicates
    for rep in range(1,201):
        #Read in inference results
        res = pd.read_csv(inferenceFile, sep='\t', header=0)


        #Read in .fixed file
        fixed = pd.read_csv(fixedFile, skiprows=2, sep=' ',
                           names=['tempID', 'permID', 'mutType', 'location', 's', 'h', 'initial_subpop', 
                                  'origin_gen','fix_gen'])

        #Keep only beneficial mutations
        sweeps = fixed[fixed.mutType=='m0']
        #Keep only fixations that occured up to 0.5Ne generations ago
        sweeps = sweeps[sweeps.fix_gen >= (17*7000)-(0.5*7000)]
        #Reset index for sweeps df
        sweeps = sweeps.reset_index(drop=True)
        #Set window start and end for sweep detection
        sweeps['win_start'] = sweeps.location - (winSize/2)
        #If start is less than 0, set to 0
        sweeps['win_start'] = np.where(sweeps.win_start < 0, 0, sweeps.win_start)
        sweeps['win_end'] = sweeps.location + (winSize/2)
        sweeps['win_end'] = np.where(sweeps.win_end > regionLen, regionLen, sweeps.win_end)
        #Create list of windows based on window size
        wins = [i for i in range(1, regionLen + winSize, winSize)]
        #Bin snps by window
        res['genomic_window'] = pd.cut(res['snp_position'], wins, labels=wins[:-1])
        #Create binary column for whether a sweep falls within range of this snp
        res['sweep'] = 0
        #Loop through sweeps, identifying snps that are within range of the sweep, and setting binary to 1
        for i in range(0, len(sweeps)):
            res['sweep'] = np.where((res.snp_position >= sweeps.win_start[i]) & 
                                        (res.snp_position <= sweeps.win_end[i]), 1, res.sweep)

        #Group by windows, get max H12 values, and whether sweeps exist, and whether thresholds are passed
        l1 = list(res.groupby('genomic_window')['H12'].max())    
        l2 = list(res.groupby('genomic_window')['sweep'].sum())
        #Create df
        df = pd.DataFrame([l2,l1]).T
        df.columns = ['sweep', 'H12']

        #Set threshold for the 95% tail and null threshold
        outlier_threshold = df['H12'].quantile(.95)
        df['95th_quantile'] = np.where(df['H12']>=outlier_threshold, 1, 0)
        df['null_threshold'] = np.where(df['H12']>=null_threshold, 1, 0)

        #Set binary columns to 1 if theyre greater than 1 (as sum was used)
        df['sweep'] = np.where(df['sweep'] > 1, 1, df['sweep'])

        #START ADJACENT WINDOWS METHOD FOR OUTLIER METHOD
        t = outlier_threshold
        #Set choices and conditions, then use np.select to create new column
        choices = ['FN','TP','FP','TN']
        conditions = [(df[col1]<t) & (df[col2]==1), 
                     (df[col1]>=t) & (df[col2]==1),
                     (df[col1]>=t) & (df[col2]==0),
                     (df[col1]<t) & (df[col2]==0)]

        df['outlier_outcome'] = np.select(conditions, choices, default=np.nan)
        lst = list(df.outlier_outcome)
        #Create columns for the final outcome (which will be adjusted through iterative process)
        df['final_outlier_outcome'] = df.outlier_outcome
        #Create columns shifted one forward and back
        df["next_outlier_outcome"] = df.outlier_outcome.shift(-1) 
        df["prior_outlier_outcome"] = df.outlier_outcome.shift(1)
        #While loop keeps until there are no instances where a true positive is next to a false positive
        while(len(df[(df.next_outlier_outcome.eq("TP") & df.final_outlier_outcome.eq("FP")) |
           (df.prior_outlier_outcome.eq("TP") & df.final_outlier_outcome.eq("FP"))]) > 0):
            #If the next window (either before or after) is a TP, set to TP
            df['final_outlier_outcome'] = np.where(df.next_outlier_outcome.eq("TP") & 
                                                   df.outlier_outcome.eq("FP") & (df[col1]>=t), 'TP', 
                                           df.final_outlier_outcome)
            df['final_outlier_outcome'] = np.where(df.prior_outlier_outcome.eq("TP") & 
                                                   df.outlier_outcome.eq("FP") & (df[col1]>=t), 'TP', 
                                           df.final_outlier_outcome)
            #Adjust shifted columns to account for changes
            df["next_outlier_outcome"] = df.final_outlier_outcome.shift(-1)
            df["prior_outlier_outcome"] = df.final_outlier_outcome.shift(1)

        #START ADJACENT WINDOWS METHOD FOR nULL METHOD
        #Repeat steps as above but for the null threshold
        t = null_threshold
        choices = ['FN','TP','FP','TN']
        conditions = [(df[col1]<t) & (df[col2]==1), 
                     (df[col1]>=t) & (df[col2]==1),
                     (df[col1]>=t) & (df[col2]==0),
                     (df[col1]<t) & (df[col2]==0)]

        df['null_outcome'] = np.select(conditions, choices, default=np.nan)

        lst = list(df.null_outcome)

        df['final_null_outcome'] = df.null_outcome
        df["next_null_outcome"] = df.null_outcome.shift(-1) 
        df["prior_null_outcome"] = df.null_outcome.shift(1)

        while(len(df[(df.next_null_outcome.eq("TP") & df.final_null_outcome.eq("FP")) |
           (df.prior_null_outcome.eq("TP") & df.final_null_outcome.eq("FP"))]) > 0):
            df['final_null_outcome'] = np.where(df.next_null_outcome.eq("TP") & 
                                                   df.null_outcome.eq("FP") & (df[col1]>=t), 'TP', 
                                           df.final_null_outcome)
            df['final_null_outcome'] = np.where(df.prior_null_outcome.eq("TP") & 
                                                   df.null_outcome.eq("FP") & (df[col1]>=t), 'TP', 
                                           df.final_null_outcome)
            df["next_null_outcome"] = df.final_null_outcome.shift(-1)
            df["prior_null_outcome"] = df.final_null_outcome.shift(1)



        #Count number of TPs etc and append to lists
        TP = len(df[(df.final_outlier_outcome=='TP')])
        TN = len(df[(df.final_outlier_outcome=='TN')])
        FP = len(df[(df.final_outlier_outcome=='FP')])
        FN = len(df[(df.final_outlier_outcome=='FN')])
        #Get number of windows in the tail
        tailWins = df['95th_quantile'].value_counts()[1]
        #Get count of number of sweeps
        totalSweeps = df['sweep'].value_counts()[1]
        #Append to list
        tail.append([TP, TN, FP, FN, tailWins, totalSweeps])
        #Repeat for null analysis
        TP = len(df[(df.final_null_outcome=='TP')])
        TN = len(df[(df.final_null_outcome=='TN')])
        FP = len(df[(df.final_null_outcome=='FP')])
        FN = len(df[(df.final_null_outcome=='FN')])
        positiveWins = df['null_threshold'].value_counts()[1]

        null.append([TP, TN, FP, FN, positiveWins, totalSweeps])

    except Exception:
        pass
#Create dataframes of sums across all replicates
tdf = pd.DataFrame(tail, columns=['TP','TN','FP','FN', 'windows_in_tail', 'totalSweeps'])
ndf = pd.DataFrame(null, columns=['TP','TN','FP','FN', 'positive_windows', 'totalSweeps'])
tdf['FN%'] = (tdf['FN']/tdf['totalSweeps'])*100
tdf['TP%'] = (tdf['TP']/tdf['windows_in_tail'])*100
tdf['FP%'] = (tdf['FP']/tdf['windows_in_tail'])*100
tdf.replace([np.inf, -np.inf], np.nan)
tdf.dropna(inplace=True)

ndf['FN%'] = (ndf['FN']/ndf['totalSweeps'])*100          
ndf['TP%'] = (ndf['TP']/ndf['positive_windows'])*100
ndf['FP%'] = (ndf['FP']/ndf['positive_windows'])*100
ndf.replace([np.inf, -np.inf], np.nan)
ndf.dropna(inplace=True)
#Append to results list
results.append([demog, N,tdf['TP%'].mean(),tdf['FP%'].mean(),tdf['FN%'].mean(),
                ndf['TP%'].mean(),ndf['FP%'].mean(),ndf['FN%'].mean()])
#Convert to df and concatenate with results df
rdf = pd.DataFrame(results, columns=['demog','2Nes','TP_tail','FP_tail','FN_tail',
                                                     'TP_null','FP_null','FN_null'])
#Output to file    
rdf.to_csv(outFile, sep='\t', header=True, index=False)